Loading required packages:


In [1]:
library(ComplexHeatmap)
library(ggplot2)
library(plotly)
library(dplyr)
library(DESeq2)
library(phyloseq)
library(tidyverse)
library(ggplot2)
library(ggrepel)
library(VennDiagram)
library(ALDEx2)
library(radEmu)
library(ggpubr)
library(ggvenn)
library(pheatmap)

Loading required package: grid

ComplexHeatmap version 2.23.1
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: https://github.com/jokergoo/ComplexHeatmap
Documentation: http://jokergoo.github.io/ComplexHeatmap-reference

If you use it in published research, please cite either one:
- Gu, Z. Complex Heatmap Visualization. iMeta 2022.
- Gu, Z. Complex heatmaps reveal patterns and correlations in multidimensional 
    genomic data. Bioinformatics 2016.


The new InteractiveComplexHeatmap package can directly export static 
complex heatmaps into an interactive Shiny app with zero effort. Have a try!

This message can be suppressed by:
  suppressPackageStartupMessages(library(ComplexHeatmap))



Attaching package: ‘plotly’


The following object is masked from ‘package:ggplot2’:

    last_plot


The following object is masked from ‘package:ComplexHeatmap’:

    add_heatmap


The following object is masked from ‘package:stats’:

    filter


The following objec

In [2]:
setwd("~/R_tasks")

In [ ]:
source_file<- read.delim("source1.txt", header= T, row.names = 1, sep = "\t")
metadata <- source_file[, (ncol(source_file) - 1):ncol(source_file)]
assay<- source_file[, -c((ncol(source_file) - 1):ncol(source_file))]
#metadata<- source_file[,c(1,2)]
#assay<- source_file[,-c(1,2)]
assay1<- t(assay)

r_a<- rownames(assay1)
assay1 <- matrix(as.numeric(assay1), nrow = nrow(assay1))
rownames(assay1)<- r_a
colnames(assay1)<- rownames(metadata)
assay1[is.na(assay1)]<-0

# RENAMING the taxa
taxa_names <- c(rownames(assay1))

# Removing unwanted characters (like "EUPATH_0009353")
taxa_cleaned <- gsub("EUPATH.*", "", taxa_names)
rownames(assay1)<-taxa_cleaned
# Splitting taxa by delimiters
taxa_split <- strsplit(taxa_cleaned, "[_.]+")

keep <- rowSums(assay1 >= 5) >= 2
assay1_filtered <- assay1[keep, ]
colnames(metadata)[colnames(metadata)=="study_condition..MBIOTEMP_study_condition."]<- "condition"

# Writting down counts and metadata after organizing them to run ANCOM-BC using QIIME2 on linux
###################
write.csv(assay1_filtered,file = "feature_table.csv")
write.csv(metadata, file = "sample_metadata.csv")
###################

assay1<-round(assay1)
assay1<- assay1+1


#this step is for designing the study to be between two conditions:
metadata$condition <- ifelse(metadata$condition == "control", "control", "Tumor")
cond<- metadata$condition

[1] TRUE

[1] "Abiotrophia.."     "Acholeplasma.."    "Acidaminococcus.."
[4] "Actinobaculum.."   "Actinomyces.."     "Adlercreutzia.."

[1] "condition"                        "study_name..MBIOTEMP_study_name."

### for ALDEx2

In [9]:
keep <- rowSums(assay1 >= 5) >= 2
assay1_filtered <- assay1[keep, ]

# Performing ALDEx2 transformation using clr normalization
aldex_data <- aldex.clr(assay1_filtered, cond, denom="iqlr", verbose=TRUE)

# Running differential abundance analysis (Wilcoxon test for two conditions)
aldex_results <- aldex.ttest(aldex_data)

# Computing effect sizes
effect_sizes <- aldex.effect(aldex_data)

# Merging results into a single dataframe
aldex_output <- data.frame(aldex_results, effect_sizes)

# Orderin by p-value
aldex_output <- aldex_output[order(aldex_output$we.ep), ]
# Computing approximate fold change
aldex_output$fold_change <- 2^aldex_output$diff.btw

# results
write.csv(aldex_output, file = "aldex_results.csv", row.names = TRUE)

conditions vector supplied

operating in serial mode

removed rows with sums equal to zero

computing iqlr centering

data format is OK

dirichlet samples complete

transformation complete

operating in serial mode

sanity check complete

rab.all  complete

rab.win  complete

rab of samples complete

within sample difference calculated

between group difference calculated

group summaries calculated

unpaired effect size calculated

summarizing output



### for radEmu:

In [ ]:
design_matrix <- radEmu::make_design_matrix(formula = ~ condition, data = metadata)
colnames(design_matrix)
k_index <- 2
otu <- otu_table(assay1, taxa_are_rows = TRUE)
samp <- sample_data(metadata)
physeq <- phyloseq(otu, samp)

physeq <- filter_taxa(physeq, function(x) sum(x >= 5) >= 2, TRUE)
test_kj <- expand.grid(
  k = k_index,
  j = 1:ntaxa(physeq)
)
ch_fit <- emuFit(
  formula = ~ condition,
  Y = physeq,
  test_kj = test_kj
)
x <- ch_fit$coef
x <- x[order(x$pval), ]
write.csv(x, "rademu_results.csv")



[1] "(Intercept)"    "conditionTumor"

In [ ]:
dds<-DESeqDataSetFromMatrix(countData = assay1,
                            colData = metadata,
                            design = ~condition)
dds$condition<-as.factor(dds$condition)
dds$condition<- relevel(dds$condition,ref = "control")
keep<- rowSums(counts(dds)>=5)>=2
dds1<- dds[keep,]
f<-counts(dds1)
dds1<-DESeq(dds1)
res<-results(dds1,alpha = 0.05)
res<-res[order(res$pvalue),]
write.csv(res,file = "primary1.csv")



converting counts to integer mode

Warning message in DESeqDataSet(se, design = design, ignoreRank):
“some variables in design formula are characters, converting to factors”


# for finding Intersected and Unique taxa:

In [11]:

resa_df<- read.csv("primary1.csv",header = T)
rownames(resa_df)<- resa_df$X
sub_resa100<-head(resa_df, 100)
sub_resa50<-head(resa_df, 25) # It defers based on the total number of filltered taxa
des100<-sub_resa100$X
des50<-sub_resa50$X

#rademu
rademu<- read.csv("rademu_results.csv",header=T)
rademu$category<-make.unique(rademu$category)
rownames(rademu)<-rademu$category
colnames(rademu)[colnames(rademu)=="pval"]<- "pvalue"
sub_rad100<- head(rademu,100 )
sub_rad50<- head(rademu, 25)
rad100<-sub_rad100$category
rad50<-sub_rad50$category

#aldex2
aldex<-read.csv("aldex_results.csv", header = T)
rownames(aldex)<-aldex$X
colnames(aldex)[colnames(aldex)=="we.ep"]<-"pvalue"
sub_ald100<- head(aldex, 100)
sub_ald50<- head(aldex, 25)
ald100<-sub_ald100$X

ald50<-sub_ald50$X


In [13]:
# Extracting row names
rows_assay1 <- rownames(assay1)
rows_resa100 <- rownames(sub_resa50)
rows_rad100 <- rownames(sub_rad50)
rows_ald100 <- rownames(sub_ald50)

# Find intersection
intersected_rows <- Reduce(intersect, list(rows_resa100, rows_rad100, rows_ald100))

assay1_intersected <- assay1[intersected_rows, ]

#########
###########
##########
head(intersected_rows)
head(rownames(assay1))
setdiff(intersected_rows, rownames(assay1)) %>% head(10)



# Find unique taxa for each tool
unique_resa100 <- setdiff(setdiff(rows_resa100, rows_ald100), rows_rad100)
sub_resa100_unique <- assay1[unique_resa100, ]

unique_rad100 <- setdiff(setdiff(rows_rad100, rows_resa100), rows_ald100)
sub_rad100_unique <- assay1[unique_rad100, ]

unique_ald100 <- setdiff(setdiff(rows_ald100, rows_resa100), rows_rad100)
sub_ald100_unique <- assay1[unique_ald100, ]

# Generate plot title
heatmap_title <- paste0(
  "Intersected taxa: ", length(intersected_rows),
  " | Unique DESeq2: ", length(unique_resa100),
  " | Unique RADEMU: ", length(unique_rad100),
  " | Unique ALDEX2: ", length(unique_ald100)
)


[1] "Veillonella.."        "Streptococcus.."      "Klebsiella.."        
[4] "Mediterraneibacter.." "Ligilactobacillus.."  "Alistipes.."

[1] "Abiotrophia.."     "Acholeplasma.."    "Acidaminococcus.."
[4] "Actinobaculum.."   "Actinomyces.."     "Adlercreutzia.."

character(0)

In [ ]:
# Intersected heatmap with title
pheatmap(
  assay1_intersected,
  cluster_rows = TRUE,
  cluster_cols = TRUE,
  width = 15,
  height = 8,
  dpi = 300,
  filename = "heatmap_assay1_intersected.png",
  main = heatmap_title
)

# Unique DESeq2
pheatmap(
  sub_resa100_unique,
  cluster_rows = TRUE,
  cluster_cols = TRUE,
  width = 15,
  height = 8,
  dpi = 300,
  filename = "heatmap_sub_des_unique.png"
)

ERROR: Error in hclust(d, method = method): must have n >= 2 objects to cluster


In [17]:
# Unique RADEMU
pheatmap(
  sub_rad100_unique,
  cluster_rows = TRUE,
  cluster_cols = TRUE,
  width = 15,
  height = 8,
  dpi = 300,
  filename = "heatmap_sub_rad_unique.png"
)

# Unique ALDEX2
pheatmap(
  sub_ald100_unique,
  cluster_rows = TRUE,
  cluster_cols = TRUE,
  width = 15,
  height = 8,
  dpi = 300,
  filename = "heatmap_sub_ald_unique.png"
)
